In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('education_merged.tsv', sep='\t', dtype=str)

In [4]:
# Remove whitespace and replace double spaces with single spaces
for col in df.columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].str.replace('  ', ' ')

In [5]:
# Clean credit card and phone - contains double spaces as a separator
cols = ['credit_card_number', 'phone']
for col in cols:
    df[col] = df[col].str.replace(' ', '')

In [6]:
# Convert all to lower case
for col in df.columns:
    df[col] = df[col].str.lower()

In [7]:
# Fix '24' in timestamp - should be 00
cols = ['employment_timestamp']
for col in cols:
    df[col] = df[col].str.replace('t24', 't00')

In [8]:
# Apply regex to determine if an email is valid or not: for this exercise, if it contains @ and at least one dot after the @
def fix_email(row):
    email = str(row['email'])
    if '@' not in email:
        try:
            pre = email.split('.com')[0]
            post = email.split('.com')[1]
            pre_dot = pre.rfind('.')
            domain = pre[pre_dot+1:]
            user = pre[:pre_dot]
            return f'{user}@{domain}.com{post}'
        except IndexError:
            return 'unknown'
    else:
        return email

In [9]:
# Convert timestamp string to a datetime object to create additional time attributes
from datetime import datetime
df['employ_ts'] = [datetime.strptime(x, '%Y-%m-%dt%H:%M%z') for x in df['employment_timestamp']]

In [10]:
df['employ_year'] = [x.year for x in df['employ_ts']]
df['employ_month'] = [x.month for x in df['employ_ts']]
df['employ_day'] = [x.day for x in df['employ_ts']]
df['employ_hour'] = [x.hour for x in df['employ_ts']]
df['employ_minute'] = [x.minute for x in df['employ_ts']]
df['employ_dt'] = [x.date() for x in df['employ_ts']]

In [11]:
# Apply email fix function above to test for valid email
df['email_fix'] = df.apply(lambda row: fix_email(row), axis=1)

In [12]:
display(df)

,rec_id,ssn,first_name,middle_name,last_name,gender,current_age,birth_date,street_address,suburb,postcode,state,phone,email,education,occupation,salary,credit_card_number,years_of_experience,employment_timestamp,employ_ts,employ_year,employ_month,employ_day,employ_hour,employ_minute,employ_dt,email_fix
0,r00002288,i120208008,suzanne,yvonne,williams,f,49,16/1/1971,133 lennox street cornet place,maryborough,4650,qld,0779994542,suzanne54@aol.com,certificate-iv,engineering-worker,68521,0069861693959066,27,2017-03-15t20:09+00:00,2017-03-15 20:09:00+00:00,2017,3,15,20,9,2017-03-15,suzanne54@aol.com
1,r00005402,a104000205,mckenzie,louise,waddell,f,30,9/6/1990,17 walker street grevillea court,crescent head,2440,nsw,0295693627,waddell67@aol.com,certificate-ii,construction-worker,51258,1443975475513789,9,2018-02-16t11:09+00:00,2018-02-16 11:09:00+00:00,2018,2,16,11,9,2018-02-16,waddell67@aol.com
2,r00008727,h180070027,harold,eugene,godfrey,m,77,15/4/1943,258 spring street yukana retirement village,toowoomba,4350,qld,0706828739,harold.godfrey@hotmail.com,certificate-iii,re-tired,0,1266345222664678,45,2020-04-14t06:28+00:00,2020-04-14 06:28:00+00:00,2020,4,14,6,28,2020-04-14,harold.godfrey@hotmail.com
3,r00008736,a100780306,james,enrique,bryant,f,22,22/2/1994,35 jubilee drive,waikiki,6169,wa,NaN,NaN,graduate-certificate,chief-executive,NaN,5867776465639458,7,2016-02-24t22:50+00:00,2016-02-24 22:50:00+00:00,2016,2,24,22,50,2016-02-24,unknown
4,r00013881,a101880013,steven,c,greenberg,f,59,24/9/1961,258 spring street yukana retirement vlge,toowoomba,4350,qld,0744145113,steven58@hotmail.com,senior-secondary-school,machinary-driver,-9999,0841430012480474,33,2013-10-15t21:45+00:00,2013-10-15 21:45:00+00:00,2013,10,15,21,45,2013-10-15,steven58@hotmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19166,r99977316,d197139679,karl,NaN,battersby,f,45,19/6/1975,4 maud street rosetta village,victor harbor,5211,sa,0896511605,battersby.karl@yahoo.com,certificate-i,other-service-worker,57925,1405897848384726,22,2016-01-13t11:07+00:00,2016-01-13 11:07:00+00:00,2016,1,13,11,7,2016-01-13,battersby.karl@yahoo.com
19167,r99978797,b187797999,wardell,lucas,lacewell,f,62,10/3/1958,21 seabrook avenue solander house,port douglas,4871,qld,0784750792,btcqkqcgqg@gmail.com,certificate-i,clerical-worker,NaN,2979491057432168,42,2019-06-05t02:49+00:00,2019-06-05 02:49:00+00:00,2019,6,5,2,49,2019-06-05,btcqkqcgqg@gmail.com
19168,r99982682,i182899926,aaron,alphonso,odom,m,73,30/6/1946,4 agnew place,armadale,6112,wa,0880637411,odom8@mail.com,advance-diploma,other-technician,91999,5101089869620641,52,2016-01-14t10:50+00:00,2016-01-14 10:50:00+00:00,2016,1,14,10,50,2016-01-14,odom8@mail.com
19169,r99983010,b198900913,timothy,samuel,shumaker,f,25,31/8/1995,670 princes highway apt 7,berwick,3806,vic,0364805172,hinnseklfj@mail.com.au,certificate-i,carer,31357,8687363058657882,6,2020-05-03t09:50+00:00,2020-05-03 09:50:00+00:00,2020,5,3,9,50,2020-05-03,hinnseklfj@mail.com.au


In [59]:
# # Validate if the suburb entered matches the postcode entered (0 = no, 1 = yes, invalid = postcode doesn't exist)
# # Where it is invalid the values can be manually updated with a lookup - but this is not done for this exercise
# # Relevance of location not important for the analysis requested - so not updated in the final dataset
# pc = pd.read_csv('australian_postcodes.csv', dtype=str)
# pc = pc[['postcode', 'locality', 'state']]
# pc1 = pc.groupby('postcode')['locality'].apply(list)
# df = df.replace(np.nan, '', regex=True)

In [60]:
# def pcode_valid(row):
#     if row['postcode'] != '' and row['suburb'] != '':
#         try:
#             pcode_src = row['postcode']
#             suburb_src = row['suburb']
#             suburbs_lookup = [x.lower() for x in pc1[pcode_src]]
#             if suburb_src in suburbs_lookup:
#                 return 1
#             else:
#                 return 0
#         except KeyError:
#             return 'invalid postcode'

In [61]:
# df['pcode_check'] = df.apply(lambda row: pcode_valid(row), axis=1)

In [62]:
# pc2 = pc
# pc2['key'] = pc.locality + pc.state
# pc2['key'] = pc2.key.str.lower()

In [63]:
# def suburb_valid(row):
#     if row['postcode'] != '' and row['suburb'] != '' and row['state'] != '':
#         pcode_src = row['postcode']
#         suburb_src = row['suburb']
#         state_src = row['state']
#         src_key = suburb_src + state_src
#         try:
#             postcode_lookup = pc2.loc[pc2['key'] == src_key]['postcode'].values[0]
#             if pcode_src == postcode_lookup:
#                 return 1
#             else:
#                 return 0
#         except IndexError:
#             return 'invalid locality'

In [64]:
# df['suburb_check'] = df.apply(lambda row: suburb_valid(row), axis=1)

In [13]:
# Convert date columns to yyyy-mm-dd for input into database
cols = ['birth_date']
for col in cols:
    df[col+'_dt'] = [datetime.strptime(x, '%d/%m/%Y') for x in df[col]]

In [14]:
df.to_csv('education_clean_3.csv', index=False)